# input perturbation

In [1]:
from sklearn import metrics
import scipy as sp
import numpy as np
import math
from sklearn import metrics

def perturbation_rank(model, x, y, names, regression):
    errors=[]
    
    for i in range(x.shape[1]):
        hold = np.array(x[:, i])
        np.random.shuffle(x[:,i])
        
        if regression:
            pred = model.predict(x)
            error = metrics.mean_squared_error(y, pred)
        else:
            pred = model.predict_proba(x)
            error = metrics.log_loss(y, pred)
        
        errors.append(error)
        x[:, i] = hold
    max_error = np.max(errors)
    importance = [e/max_error for e in errors]
    
    data = {'name': names, 'error': errors, 'importance': importance}
    result = pd.DataFrame(data, columns = ['name', 'error','importance'])
    result.sort_values(by=['importance'], ascending = [0], inplace=True)
    result.reset_index(inplace=True, drop=True)
    return result

In [2]:
import pandas as pd
import io
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/iris.csv", 
    na_values=['NA', '?'])
#convert to numpy -Classification
x=df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values
dummies = pd.get_dummies(df['species']) #classification
species = dummies.columns
y = dummies.values

#split train/test
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size=0.25, random_state=42)

#build neural network

model=Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) #hidden 1
model.add(Dense(25, activation='relu')) #hidden 2
model.add(Dense(y.shape[1], activation='softmax')) #output
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(x_train, y_train, verbose=2, epochs=100)

Epoch 1/100
4/4 - 2s - loss: 1.1457
Epoch 2/100
4/4 - 0s - loss: 1.0627
Epoch 3/100
4/4 - 0s - loss: 0.9881
Epoch 4/100
4/4 - 0s - loss: 0.9296
Epoch 5/100
4/4 - 0s - loss: 0.8787
Epoch 6/100
4/4 - 0s - loss: 0.8302
Epoch 7/100
4/4 - 0s - loss: 0.7878
Epoch 8/100
4/4 - 0s - loss: 0.7484
Epoch 9/100
4/4 - 0s - loss: 0.7155
Epoch 10/100
4/4 - 0s - loss: 0.6883
Epoch 11/100
4/4 - 0s - loss: 0.6602
Epoch 12/100
4/4 - 0s - loss: 0.6373
Epoch 13/100
4/4 - 0s - loss: 0.6143
Epoch 14/100
4/4 - 0s - loss: 0.5950
Epoch 15/100
4/4 - 0s - loss: 0.5768
Epoch 16/100
4/4 - 0s - loss: 0.5598
Epoch 17/100
4/4 - 0s - loss: 0.5434
Epoch 18/100
4/4 - 0s - loss: 0.5286
Epoch 19/100
4/4 - 0s - loss: 0.5150
Epoch 20/100
4/4 - 0s - loss: 0.5017
Epoch 21/100
4/4 - 0s - loss: 0.4909
Epoch 22/100
4/4 - 0s - loss: 0.4771
Epoch 23/100
4/4 - 0s - loss: 0.4651
Epoch 24/100
4/4 - 0s - loss: 0.4554
Epoch 25/100
4/4 - 0s - loss: 0.4438
Epoch 26/100
4/4 - 0s - loss: 0.4339
Epoch 27/100
4/4 - 0s - loss: 0.4254
Epoch 28/1

In [11]:
pred = model.predict(x_test)
predict_classes = np.argmax(pred, axis=1)
expected_classes = np.argmax(y_test, axis=1)
correct =metrics.accuracy_score(expected_classes, predict_classes)
print(f'accuracy:{correct}')

accuracy:1.0


In [20]:
# rank the features
names = list(df.columns) # x+y column names
names.remove('species') #remove target y
rank = perturbation_rank(model, x_test, y_test, names, False)
rank

C:\Users\cheny\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\cheny\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\cheny\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\cheny\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.

,name,error,importance
0,petal_l,3.773998,1.000000
1,petal_w,0.664681,0.176121
2,sepal_l,0.111667,0.029588
3,sepal_w,0.097022,0.025708
